In [1]:
%load_ext autoreload
%autoreload 2

# Power point ACTOR-CRITIC
https://docs.google.com/presentation/d/1vub5elAjRyMpzrhiOcOoqCzmqh0VMn-br5Mr4AU53GE/edit?usp=sharing

In [2]:
from REINFORCE_helper import RunningVariance
from time import time
from REINFORCE_helper import BaseAgent
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
import keras.backend as K
import numpy as np

Using TensorFlow backend.


In [3]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)
import moviepy.editor as mpy

In [4]:
class ReinforceAgent(BaseAgent):
    # def __init__(self):
    def get_policy_model(self, lr=0.001, hidden_layer_neurons = 128, input_shape=[4], output_shape=2):
        ## Defino métrica - loss sin el retorno multiplicando
        def loss_metric(y_true, y_pred):
            y_true_norm = K.sign(y_true)
            return K.categorical_crossentropy(y_true_norm, y_pred)
        model = Sequential()
        model.add(Dense(hidden_layer_neurons, input_shape=input_shape, activation='relu'))
        model.add(Dense(output_shape, activation='softmax'))
        ## Por que la categorical_crossentropy funciona ok?
        model.compile(Adam(lr), loss=['categorical_crossentropy'], metrics=[loss_metric])
        return model
    
    def get_action(self, eval=False):
        p = self.model.predict([self.observation.reshape(1, self.nS)])
        if eval is False:
            action = np.random.choice(self.nA, p=p[0]) #np.nan_to_num(p[0])
        else:
            action = np.argmax(p[0])
        action_one_hot = np.zeros(self.nA)
        action_one_hot[action] = 1
        return action, action_one_hot, p
    
    def get_entropy(self, preds, epsilon=1e-12):
        entropy = np.mean(-np.sum(np.log(preds+epsilon)*preds, axis=1)/np.log(self.nA))
        return entropy
    
    def get_discounted_rewards(self, r):
        # Por si es una lista
        r = np.array(r, dtype=float)
        """Take 1D float array of rewards and compute discounted reward """
        discounted_r = np.zeros_like(r)
        running_add = 0
        for t in reversed(range(0, r.size)):
            running_add = running_add * self.gamma + r[t]
            discounted_r[t] = running_add
        return discounted_r 

In [7]:
# reinforce_agent = ReinforceAgent('CartPole-v1', n_experience_episodes=1, EPISODES=500, eval_period=20, epochs=1, lr=0.001, algorithm='REINFORCE_CAUSAL',
#                                  fps=20, gif_to_board=True)
# reinforce_agent = ReinforceAgent('Acrobot-v1', n_experience_episodes=1, EPISODES=2000, epochs=1, lr=0.001, algorithm='REINFORCE_CAUSAL', fps=25,
#                              gif_to_board=True)
reinforce_agent = ReinforceAgent('LunarLander-v2', n_experience_episodes=50, EPISODES=2000, epochs=5, lr=0.0001, algorithm='REINFORCE_CAUSAL', 
                                 gif_to_board=True)

initial_time = time()
running_variance = RunningVariance()


while reinforce_agent.episode < reinforce_agent.EPISODES:
    obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs = reinforce_agent.get_experience_episodes()
    
    for dr in disc_sum_rews:
        running_variance.add(dr)

    pseudolabels = actions*disc_sum_rews.reshape(-1, 1)

    history = reinforce_agent.model.fit(obs, pseudolabels, verbose=0, epochs=reinforce_agent.epochs, batch_size=128)
    
    reinforce_agent.log_data(reinforce_agent.episode, 
                      history.history['loss'][0], 
                      np.mean(ep_len), 
                      reinforce_agent.get_entropy(preds), 
                      running_variance.get_variance(), 
                      history.history['loss_metric'][0], 
                      time() - initial_time, np.mean(ep_returns[-1]))
    
reinforce_agent.writer.close()

correr en linea de comando: tensorboard --logdir logs/
Episode: 51

t:   0%|          | 0/86 [00:00<?, ?it/s, now=None]


Model on episode 52 improved from -inf to -254.59669949456688. Saved!
MoviePy - Building file /tmp/tmpdotaelwr.gif with imageio.


Episode: 311

t:   0%|          | 0/80 [00:00<?, ?it/s, now=None]


Model on episode 312 improved from -254.59669949456688 to -110.434214025497. Saved!
MoviePy - Building file /tmp/tmpad_18kem.gif with imageio.


Episode: 571

t:   0%|          | 0/82 [00:00<?, ?it/s, now=None]


Model on episode 572 improved from -110.434214025497 to -52.37121532736121. Saved!
MoviePy - Building file /tmp/tmpfgaseutn.gif with imageio.


Episode: 2027